# カメラ画像取得

In [1]:
import cv2

# カメラのキャプチャオブジェクトを作成（デフォルトのカメラを使用）
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("カメラを開くことができませんでした")
    exit()

while True:
    # フレームを取得
    ret, frame = cap.read()

    if not ret:
        print("フレームの取得に失敗しました")
        break

    # フレームを表示
    cv2.imshow('Camera', frame)

    # 'q'キーを押したらループを終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# キャプチャを解放し、ウィンドウを閉じる
cap.release()
cv2.destroyAllWindows()

# 円検出及び直径計算

In [3]:
import cv2
import numpy as np

# カメラのキャプチャオブジェクトを作成（デフォルトのカメラを使用）
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("カメラを開くことができませんでした")
    exit()

while True:
    # フレームを取得
    ret, frame = cap.read()

    if not ret:
        print("フレームの取得に失敗しました")
        break

    # グレースケールに変換
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # ノイズを減らすためにぼかし処理を適用
    gray = cv2.medianBlur(gray, 5)

    # Hough変換で円を検出
    circles = cv2.HoughCircles(
        gray, 
        cv2.HOUGH_GRADIENT, 
        dp=1.2, 
        minDist=100, 
        param1=100, 
        param2=30, 
        minRadius=0, 
        maxRadius=0
    )

    # 円が検出された場合
    if circles is not None:
        # 結果を丸めて整数に変換
        circles = np.uint16(np.around(circles))

        for i in circles[0, :]:
            # 円の中心に円を描画
            cv2.circle(frame, (i[0], i[1]), 1, (0, 100, 100), 3)
            # 円周に円を描画
            cv2.circle(frame, (i[0], i[1]), i[2], (255, 0, 255), 3)
            # 円の半径を表示
            cv2.putText(frame, f"Diameter: {2*i[2]} px", (i[0] - 50, i[1] - i[2] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # フレームを表示
    cv2.imshow('Detected Circles', frame)

    # 'q'キーを押したらループを終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# キャプチャを解放し、ウィンドウを閉じる
cap.release()
cv2.destroyAllWindows()

C:\Users\Yoshiyuki Hatta\AppData\Local\Temp\ipykernel_8132\3567665988.py:48: RuntimeWarning: overflow encountered in scalar subtract
  cv2.putText(frame, f"Radius: {2*i[2]} px", (i[0] - 50, i[1] - i[2] - 10),


# 計測結果のファイル出力

In [4]:
import cv2
import numpy as np
import csv
import os

# 出力するCSVファイル名を指定
csv_file = "circle_radii.csv"

# CSVファイルが既に存在する場合は削除する
if os.path.exists(csv_file):
    os.remove(csv_file)

# CSVファイルを開いて書き込みモードで準備
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Frame", "Circle", "Diameter (px)"])  # ヘッダー行の書き込み

    # カメラのキャプチャオブジェクトを作成（デフォルトのカメラを使用）
    cap = cv2.VideoCapture(1)

    if not cap.isOpened():
        print("カメラを開くことができませんでした")
        exit()

    frame_count = 0

    while True:
        # フレームを取得
        ret, frame = cap.read()

        if not ret:
            print("フレームの取得に失敗しました")
            break

        frame_count += 1

        # グレースケールに変換
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # ノイズを減らすためにぼかし処理を適用
        gray = cv2.medianBlur(gray, 5)

        # Hough変換で円を検出
        circles = cv2.HoughCircles(
            gray, 
            cv2.HOUGH_GRADIENT, 
            dp=1.2, 
            minDist=100, 
            param1=100, 
            param2=30, 
            minRadius=0, 
            maxRadius=0
        )

        # 円が検出された場合
        if circles is not None:
            # 結果を丸めて整数に変換
            circles = np.uint16(np.around(circles))

            circle_count = 0

            for i in circles[0, :]:
                circle_count += 1
                # 円の中心に円を描画
                cv2.circle(frame, (i[0], i[1]), 1, (0, 100, 100), 3)
                # 円周に円を描画
                cv2.circle(frame, (i[0], i[1]), i[2], (255, 0, 255), 3)
                # 円の半径を表示
                cv2.putText(frame, f"Diameter: {2*i[2]} px", (i[0] - 50, i[1] - i[2] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

                # 検出された円の半径をCSVに書き込む
                writer.writerow([frame_count, circle_count, 2*i[2]])

        # フレームを表示
        cv2.imshow('Detected Circles', frame)

        # 'q'キーを押したらループを終了
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # キャプチャを解放し、ウィンドウを閉じる
    cap.release()
    cv2.destroyAllWindows()

print(f"直径のデータを {csv_file} に保存しました。")

半径のデータを circle_radii.csv に保存しました。
